<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Study-predictions" data-toc-modified-id="Study-predictions-1">Study predictions</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Load-model-and-data" data-toc-modified-id="Load-model-and-data-1.3">Load model and data</a></span></li><li><span><a href="#Predictions" data-toc-modified-id="Predictions-1.4">Predictions</a></span></li><li><span><a href="#Current-metrics" data-toc-modified-id="Current-metrics-1.5">Current metrics</a></span></li></ul></li></ul></div>

# Study predictions

## Goal

The goal of this notebook is to study predictions of a model on the validation set with the aim to improve metrics or losses.

## Imports

In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [9]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import luxai.cunet
from luxai.data import combine_data_for_training, load_match

In [10]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Load model and data

In [11]:
model = keras.models.load_model('/mnt/hdd0/Kaggle/luxai/models/42_curriculum_learning_with_best_stages/seed0/stage4/best_val_loss_model.h5', compile=False)

In [12]:
df = pd.read_csv('/mnt/hdd0/Kaggle/luxai/models/42_curriculum_learning_with_best_stages/seed0/stage4/val.csv')
df.head()

,Id,EpisodeId,Index,Reward,State,SubmissionId,InitialConfidence,InitialScore,UpdatedConfidence,UpdatedScore,FinalScore,Team
0,75076972,29991187,0,890041.0,2,23297953,35.00000,1957.279341,35.000000,1959.003122,2021.185199,Toad Brigade
1,75988959,30447180,1,1310051.0,2,23297953,35.00000,2011.593075,35.000000,2013.044434,2021.185199,Toad Brigade
2,72070727,28487063,1,230012.0,2,23297953,42.79739,1466.705040,42.321043,1472.138593,2021.185199,Toad Brigade
3,74444543,29674972,1,1030042.0,2,23297953,35.00000,1932.343041,35.000000,1933.920986,2021.185199,Toad Brigade
4,73255322,29079361,0,310016.0,2,23297953,35.00000,1819.460414,35.000000,1821.367935,2021.185199,Toad Brigade


In [15]:
def load_data(df):
    matches = []
    for episode_id, player in tqdm(zip(df.EpisodeId, df.Index), total=len(df), desc='Loading matches'):
        matches.append(load_match(episode_id, player, '', '/home/gbarbadillo/luxai_ssd/matches_npz_v2'))
    return combine_data_for_training(matches)

In [16]:
data = load_data(df.loc[:5])

Loading matches:   0%|          | 0/6 [00:00<?, ?it/s]

Inputs shapes [(2157, 32, 32, 28), (2157, 1, 12)]
Outputs shapes [(2157, 32, 32, 11), (2157, 32, 32, 4)]


## Predictions

In [17]:
preds = model.predict(data[0], batch_size=32)

2021-11-23 17:27:20.035251: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


## Current metrics

In [30]:
for idx, name in enumerate(['unit', 'city']):
    print('%s masked_focal_loss: %.3f' % (name, luxai.cunet.masked_focal_loss(data[1][idx], preds[idx], zeta=1, true_weight=32).numpy()))
    print('%s masked_error: %.3f' % (name, luxai.cunet.masked_error(data[1][idx], preds[idx]).numpy()))
    print('%s true_positive_error: %.3f' % (name, luxai.cunet.true_positive_error(data[1][idx], preds[idx]).numpy()))
    print('%s true_negative_error: %.3f' % (name, luxai.cunet.true_negative_error(data[1][idx], preds[idx]).numpy()))
    print()

unit masked_focal_loss: 0.154
unit masked_error: 0.074
unit true_positive_error: 0.004
unit true_negative_error: 0.008

city masked_focal_loss: 0.016
city masked_error: 0.006
city true_positive_error: 0.007
city true_negative_error: 0.002



This values correspond to the ones we can see with tensorboard at epoch 6 of `/mnt/hdd0/Kaggle/luxai/models/42_curriculum_learning_with_best_stages/seed0/stage4`